In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('./data/crawling/preprocessed/Lv.1_dongchan.csv',encoding='euc-kr',index_col=0)
df_boolean = pd.read_csv('./data/kospi/preprocessed/df_boolean.csv',index_col=0)
df = df.drop(['str_tokens','url'],axis=1)

In [3]:
def preprocessing(document): #특수문자 기본 전처리 함수
    docu = []
    for doc in document:
        doc = re.sub('[^가-힣0-9\s.]','',doc)
        doc = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',doc)
#         doc = re.sub('[0-9]+','num',doc) #숫자를 special token num으로 치환
        pattern = re.compile(r'\s+') #중복띄어쓰기 제거
        doc = re.sub(pattern,' ',doc)
        doc = doc.replace('\xa0','') 
        doc = doc.replace('\n','')
        docu.append(doc)

    return docu

In [4]:
nan_idx = [x for x in df[df.str.isnull() ==True].index]
df = df.drop(nan_idx,axis=0) #str 결측치 삭제
df.rmk = df.rmk.fillna('nan')
df = df.reset_index(drop=True)

str_ls = [x for x in df['str']]
corrected_ls = preprocessing(str_ls)
df['str'] = corrected_ls

크롤링한 LV1 data(kospi200 한 번이라도 편입되었던 309개 기업의 텍스트)에 대한 토큰화 이전 기본 전처리를 하고 결측치 처리해주었다.

종목코드는 같지만 거래소에 공시되어 있는 기업의 이름과 dart에서 크롤링한 기업의 이름이 상충되는 것이 있어서 이를 거래소 이름으로 통일하여 주었다. 여기서 지수 편입 퇴출 효과를 제거하기 위해 10년동안 한번도 퇴출된 적이 없었던 113개의 기업들로 데이터를 다시 구성하였다.

In [5]:
change_dic = {'케이씨씨':'KCC',
 '케이티':'KT',
'케이티앤지':'KT&G',
'포스코':'POSCO',
'금호석유화학':'금호석유',
 '기아자동차':'기아차',
'롯데칠성음료':'롯데칠성',
 '삼성화재해상보험':'삼성화재',
 '쌍용양회공업': '쌍용양회',
'아모레퍼시픽그룹':'아모레G',
 '한국단자공업':'한국단자',
'한국전력공사':'한국전력',
'현대엘리베이터':'현대엘리베이',
 '현대자동차':'현대차'}


new_crp_ls = []
for i in df.crp_nm:
    for x in change_dic.keys():
        if i == x:
            i = change_dic[x]
    new_crp_ls.append(i)

df.crp_nm = new_crp_ls

symbols = df_boolean.iloc[0,:] 
con_ls = [df[df.crp_nm == i] for i in symbols]
df_all = pd.concat(x for x in con_ls).reset_index(drop=True) #10년동안 편입되어있던 113개의 기업

In [6]:
df_all

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,str
0,삼양홀딩스,70,2018,사업보고서 (2018.12),20190401004413,20190401,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 본 경영진단의견서...
1,삼양홀딩스,70,2017,사업보고서 (2017.12),20180402004217,20180402,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 본 경영진단의견서...
2,삼양홀딩스,70,2016,사업보고서 (2016.12),20170331003683,20170331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 본 경영진단의견...
3,삼양홀딩스,70,2015,사업보고서 (2015.12),20160330002684,20160330,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 본 경영진단의견...
4,삼양홀딩스,70,2014,사업보고서 (2014.12),20150331002399,20150331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 본 경영진단의견...
...,...,...,...,...,...,...,...,...
1112,KB금융,105560,2013,사업보고서 (2013.12),20140331002917,20140331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에서...
1113,KB금융,105560,2012,사업보고서 (2012.12),20130329001808,20130329,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에서...
1114,KB금융,105560,2011,사업보고서 (2011.12),20120330002209,20120330,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에서...
1115,KB금융,105560,2010,사업보고서 (2010.12),20110331002162,20110331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에서...


In [7]:
df.to_csv('./data/crawling/preprocessed/LV1_kospi200_any.csv')
df_all.to_csv('./data/crawling/preprocessed/LV1_kospi200_all.csv')